In [1]:
import typing
import math
import random

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import Tensor

In [2]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "N/A")
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

Torch version: 2.5.1
CUDA available: True
CUDA version: 12.4
Current device: 0
Device name: NVIDIA GeForce RTX 3060


In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using CUDA')
else:
    device = torch.device('cpu')
    print('Using CPU')

Using CUDA


In [4]:
torch.manual_seed(1337)

In [5]:
with open('tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
len(text)

1115393

In [7]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars = sorted(list(set(text)))
print(str().join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
vocab_size = len(chars)
vocab_size

65

In [10]:
stoi = {ch: i for i, ch in enumerate(chars)}
print(stoi)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [11]:
itos = {i: ch for ch, i in stoi.items()}
print(itos)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [12]:
def encode(s: str) -> typing.List[int]:
    return [stoi[c] for c in s]

def decode(ints: typing.List[int]) -> str:
    return str().join(itos[i] for i in ints)

In [13]:
print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [14]:
encoded_text = encode(text)
len(encoded_text)

1115393

In [15]:
data = torch.tensor(encoded_text, dtype=torch.long, device=device)
data.shape

torch.Size([1115393])

In [16]:
data.dtype

torch.int64

In [17]:
data[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [18]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f'Training size: {len(train_data)}, Validation size: {len(val_data)}')

Training size: 1003853, Validation size: 111540


In [19]:
block_size = 8 # Also called "context length"

In [20]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')

In [21]:
xb = train_data[:block_size]
yb = train_data[1:block_size+1]
print('--- As characters ---')
for t in range(block_size):
    context = xb[:t+1]
    target = yb[t]
    print(f'When the input is {decode(context.tolist())} the next character is {itos[target.item()]}')
print('--- Encoded ---')
for t in range(block_size):
    context = xb[:t+1]
    target = yb[t]
    print(f'When the input is {context} the next character is {target}')

--- As characters ---
When the input is F the next character is i
When the input is Fi the next character is r
When the input is Fir the next character is s
When the input is Firs the next character is t
When the input is First the next character is  
When the input is First  the next character is C
When the input is First C the next character is i
When the input is First Ci the next character is t
--- Encoded ---
When the input is tensor([18], device='cuda:0') the next character is 47
When the input is tensor([18, 47], device='cuda:0') the next character is 56
When the input is tensor([18, 47, 56], device='cuda:0') the next character is 57
When the input is tensor([18, 47, 56, 57], device='cuda:0') the next character is 58
When the input is tensor([18, 47, 56, 57, 58], device='cuda:0') the next character is 1
When the input is tensor([18, 47, 56, 57, 58,  1], device='cuda:0') the next character is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15], device='cuda:0') the next char

In [22]:
def get_batch(dataset: Tensor, batch_size: int, block_size: int, device=None) -> typing.Tuple[Tensor, Tensor]:
    '''
    Gets a batch of `batch_size` examples from `dataset`. Each example will
    consist of `block_size` characters. The inputs and labels will both be
    returned, both of which will be of size `(batch_size, block_size)`.
    '''

    ix = torch.randint(low=0, high=len(dataset)-block_size, size=(batch_size,), device=device)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [23]:
batch_size = 4
xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

torch.Size([4, 8])
tensor([[35, 56, 43, 52, 41, 46,  1, 59],
        [56, 50, 47, 49, 43,  1, 44, 39],
        [13,  1, 50, 47, 58, 58, 50, 43],
        [51,  6,  1, 47, 44,  1, 51, 63]], device='cuda:0')
torch.Size([4, 8])
tensor([[56, 43, 52, 41, 46,  1, 59, 54],
        [50, 47, 49, 43,  1, 44, 39, 58],
        [ 1, 50, 47, 58, 58, 50, 43,  1],
        [ 6,  1, 47, 44,  1, 51, 63,  1]], device='cuda:0')


In [24]:
for b in range(batch_size):
    print(f'Example {b}')
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'Block {t}: When the input is {context} the next character is {target}')

Example 0
Block 0: When the input is tensor([35], device='cuda:0') the next character is 56
Block 1: When the input is tensor([35, 56], device='cuda:0') the next character is 43
Block 2: When the input is tensor([35, 56, 43], device='cuda:0') the next character is 52
Block 3: When the input is tensor([35, 56, 43, 52], device='cuda:0') the next character is 41
Block 4: When the input is tensor([35, 56, 43, 52, 41], device='cuda:0') the next character is 46
Block 5: When the input is tensor([35, 56, 43, 52, 41, 46], device='cuda:0') the next character is 1
Block 6: When the input is tensor([35, 56, 43, 52, 41, 46,  1], device='cuda:0') the next character is 59
Block 7: When the input is tensor([35, 56, 43, 52, 41, 46,  1, 59], device='cuda:0') the next character is 54
Example 1
Block 0: When the input is tensor([56], device='cuda:0') the next character is 50
Block 1: When the input is tensor([56, 50], device='cuda:0') the next character is 47
Block 2: When the input is tensor([56, 50, 47

In [25]:
class BigramLanguageModel(nn.Module):

    vocab_size: int
    token_embedding_table: nn.Embedding

    def __init__(self, vocab_size: int, device=None):
        super().__init__()
        self.vocab_size = vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size, device=device)

    def forward(self, idx: Tensor, targets: typing.Optional[Tensor] = None) -> typing.Tuple[Tensor, typing.Optional[Tensor]]:
        # `idx` and targets are (B,T) tensors (batch size by time). In this case
        # 'time' represents block size.
        #
        # `logits` are (B,T,C) tensors, (batch size by time by channel), where
        # the channel dimension comes from the embedding table. Essentially,
        # each character in idx is replaced by an embedding vector of length C
        # (which is the vocabulary size in this case).
        logits = self.token_embedding_table(idx)
        logits = typing.cast(Tensor, logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets)

        # If `targets` was not provided, then output `logits` is a 3D tensor of
        # shape:
        #     `(batch_size, block_size, vocab_size)`
        #
        # Otherwise, if `targets` was provided, then output `logits` is a 2D
        # tensor of shape:
        #     `(batch_size * block_size, vocab_size)`
        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # `idx` is (B,T), which is `(batch_size, block_size)`
        for _ in range(max_new_tokens):
            # `logits` is (B,T,C), where C is the channel length (length of
            # embedding vector, in this case it is `vocab_length`)
            logits, loss = self(idx)
            # Get last character of logits - becomes (B, C)
            logits = logits[:, -1, :]
            # Still (B,C)
            probs = F.softmax(logits, dim=1)
            # Now its (B,1) since we are getting only one sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append sampled index to the running sequence - becomes (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)
        # The final `idx` tensor will be of shape
        #     `(batch_size, block_size + max_steps)`
        return idx

In [26]:
model = BigramLanguageModel(vocab_size, device=device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5682, device='cuda:0', grad_fn=<NllLossBackward0>)


In [27]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=100)[0].tolist()
next_str = decode(next_idx)
print(next_str)


fCxBDkL-k
zc.wfNZHxO Fn,yRtK
axxP;CkPBbABXGeCXSvgO-3 SMmd?Ya3a
hX:Y?XLtp&jjuHqUo,Kv.tbyr dXp!FZaLeWj


In [28]:
@torch.no_grad()
def estimate_loss(model: BigramLanguageModel, train_dataset: Tensor, val_dataset: Tensor, eval_iterations: int, batch_size: int, block_size: int, device = None) -> typing.Dict[str, torch.types.Number]:
    dataset_splits = {'train': train_dataset, 'val': val_dataset}
    out = dict()
    for split_name, split_dataset in dataset_splits.items():
        losses = torch.zeros(eval_iterations, device=device)
        for i in range(eval_iterations):
            xb, yb = get_batch(split_dataset, batch_size, block_size, device)
            logits, loss = model(xb, yb)
            losses[i] = loss.item()
        out[split_name] = losses.mean().item()
    return out

In [29]:
batch_size = 32
max_steps = 1000
learning_rate = 1e-3
eval_iterations = 300

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.train()

for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    model.eval()
    if max_steps < 25 or step % (max_steps // 25) == 0:
        loss_dict = estimate_loss(model, train_data, val_data, eval_iterations, batch_size, block_size, device)
        print(f'Step: {step:<7}, last seen loss: {loss.item():.4f}, estimated training loss: {loss_dict["train"]:.4f}, estimated validation loss: {loss_dict["val"]:.4f}')

    model.train()

Step: 0      , last seen loss: 4.7392, estimated training loss: 4.6838, estimated validation loss: 4.6990
Step: 40     , last seen loss: 4.6491, estimated training loss: 4.6359, estimated validation loss: 4.6494
Step: 80     , last seen loss: 4.6633, estimated training loss: 4.5871, estimated validation loss: 4.5923
Step: 120    , last seen loss: 4.5205, estimated training loss: 4.5382, estimated validation loss: 4.5428
Step: 160    , last seen loss: 4.4845, estimated training loss: 4.4868, estimated validation loss: 4.5017
Step: 200    , last seen loss: 4.4352, estimated training loss: 4.4415, estimated validation loss: 4.4558
Step: 240    , last seen loss: 4.3763, estimated training loss: 4.3950, estimated validation loss: 4.4117
Step: 280    , last seen loss: 4.3219, estimated training loss: 4.3519, estimated validation loss: 4.3666
Step: 320    , last seen loss: 4.3645, estimated training loss: 4.3121, estimated validation loss: 4.3207
Step: 360    , last seen loss: 4.2039, estimat

In [30]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=1000)[0].tolist()
next_str = decode(next_idx)
print(next_str)


F3GAbfin:F?z-TarAqwelsKath, tstVvpmXzeM?BIRo: 
adiWjhrtixOukevFrwlslji.Vy Ubruee:F'uHKdzva HNOwWtaPW&Itn,wmIpp
PFMqQ
&gxanu'oIV wQRaMEFahkHZy Tr Vl
ikdrykff xjMuepI?KKJLey'stsuGFtvXop!G$L:yum:XFNy  w3noevCXXjPU? omYaVjAw.e  dx-I
BAiERthDGINhMEsNAwhuHDwoG;G!gBADz.
DYa
EzFZvgEQRTA$NJo.HiurqUzrseCO 
qwHzxkO'NIaoW&xH:?zXrUMyme
sREPw?zppW$Z taVL&d-PIUzMEhASChrd?O:oGPuq'xFovKUonGSBBkChDfYAsQkhfjj&aEfoh!qow3C;XqWBlcvHNmmvFZ&?$C,QKsz.fa?qpQD?Affoi
LgQ3?JFrrCtgBO'GF:SrV
h'gmes$BIVEcMvKaxx
I!G
K.-RP$z$sboTnoEReVsQCS?! Jmj&IxNI!qWGAynuHDqIire oFinJmN:pMcPN,fGSoda j&jJd'edS-EDsjZFe ms,C,y IbLhsZGV3 Tu.XLwWj&CSexGMpKNZvy,yumerDiZbbPY,Kri's'? -H ax,VlK$xILA-RUo FcpP'AfhPlad
Eshr -EfC.ptCX:u,BX
I&T:R'TXrutTvyolmxOmyW;llre,GFfCQxFq&jchDWi
wndO?XGUtfC,R?vHD3LnohQju,wz.d?ZjFZzqWHr xke Tl;,KvWC&egW;fTndTuHvxaihkVvKXU,woFgiucd!p GAftW3?Ve
:d;GaEFZai'3ogERbFy?zJN wmz:
BBythN?NgaVGvpOE.!HAve vfhK?SffC'sb&BQkqglap..IW,foCe,KrdunheYbkDUNSrWB$.
NSO:hNCSVR?ee poIVj;re-a;ZADgkDMENVcMqt3yresRKhraiuC.
I.
TSpJx-PA

In [31]:
class BigramLanguageModel(nn.Module):

    vocab_size: int
    n_embd: int
    block_size: int

    token_embedding_table: nn.Embedding
    position_embedding_table: nn.Embedding
    lm_head: nn.Linear

    def __init__(self, vocab_size: int, block_size: int, n_embd: int, device=None):
        super().__init__()
        self.vocab_size = vocab_size
        self.n_embd = n_embd
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)
        self.lm_head = nn.Linear(n_embd, vocab_size, device=device)

    def forward(self, idx: Tensor, targets: typing.Optional[Tensor] = None) -> typing.Tuple[Tensor, typing.Optional[Tensor]]:
        # `idx` and targets are (B,T) tensors (batch size by time). In this case
        # 'time' represents block size.
        B, T = idx.shape

        # `logits` are (B,T,C) tensors, (batch size by time by channel), where
        # the channel dimension comes from the embedding table. Essentially,
        # each character in idx is replaced by an embedding vector of length C
        # (which is the number of embeddings in this case).
        token_embeddings = self.token_embedding_table(idx)

        # Shape is (T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device))

        # Addition gets broadcasted, shape is (B,T,C)
        x = token_embeddings + position_embeddings

        # We then apply the linear layer, which gives us a (B, T, vocab_size)
        # tensor, which are our logits.
        logits = self.lm_head(x)

        logits = typing.cast(Tensor, logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets)

        # If `targets` was not provided, then output `logits` is a 3D tensor of
        # shape:
        #     `(batch_size, block_size, vocab_size)`
        #
        # Otherwise, if `targets` was provided, then output `logits` is a 2D
        # tensor of shape:
        #     `(batch_size * block_size, vocab_size)`
        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # `idx` is (B,T), which is `(batch_size, block_size)`
        for _ in range(max_new_tokens):
            # Crop idx to the last `block_size` tokens
            idx_cond = idx[:, -block_size:]
            # `logits` is (B,T,C), where C is the channel length (length of
            # embedding vector, in this case it is `vocab_length`)
            logits, loss = self(idx_cond)
            # Get last character of logits - becomes (B, C)
            logits = logits[:, -1, :]
            # Still (B,C)
            probs = F.softmax(logits, dim=-1)
            # Now its (B,1) since we are getting only one sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append sampled index to the running sequence - becomes (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)
        # The final `idx` tensor will be of shape
        #     `(batch_size, block_size + max_steps)`
        return idx

In [32]:
batch_size = 32
max_steps = 1000
learning_rate = 1e-3
eval_iterations = 300
n_embd = 32

model = BigramLanguageModel(vocab_size, block_size, n_embd, device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.train()

for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    model.eval()
    if max_steps < 25 or step % (max_steps // 25) == 0:
        loss_dict = estimate_loss(model, train_data, val_data, eval_iterations, batch_size, block_size, device)
        print(f'Step: {step:<7}, last seen loss: {loss.item():.4f}, estimated training loss: {loss_dict["train"]:.4f}, estimated validation loss: {loss_dict["val"]:.4f}')

    model.train()

Step: 0      , last seen loss: 4.4113, estimated training loss: 4.3628, estimated validation loss: 4.3542
Step: 40     , last seen loss: 3.7722, estimated training loss: 3.8727, estimated validation loss: 3.8751
Step: 80     , last seen loss: 3.5283, estimated training loss: 3.5259, estimated validation loss: 3.5272
Step: 120    , last seen loss: 3.2791, estimated training loss: 3.3012, estimated validation loss: 3.2882
Step: 160    , last seen loss: 3.1732, estimated training loss: 3.1365, estimated validation loss: 3.1328
Step: 200    , last seen loss: 3.2168, estimated training loss: 3.0268, estimated validation loss: 3.0201
Step: 240    , last seen loss: 2.8334, estimated training loss: 2.9572, estimated validation loss: 2.9536
Step: 280    , last seen loss: 3.0127, estimated training loss: 2.8974, estimated validation loss: 2.8855
Step: 320    , last seen loss: 2.8349, estimated training loss: 2.8468, estimated validation loss: 2.8415
Step: 360    , last seen loss: 2.7042, estimat

In [33]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=1000)[0].tolist()
next_str = decode(next_idx)
print(next_str)


Be wand y dpes, d f ameGAddoounouuthe f t:
I fsthee ayre T, be mereowhe
Coo, we

Hery liso.

 s touad t wd dsts,yNothe agswodshorihevothitacushetweeouourd tO a; gartho rer pe poirined,

Foopnd duyirotid aih k y chetoIe t t an he soTuve pres by atacil h.
Whe whrir he dord byOun' romosb be tlan wd herenrupld as s-whertyouous hadnelisoshanpeseyontch:
;yoyive w, llineroreuto ghouokqketh hatoraslthe Vir bd:

LA:
AW:

xwe we
,
BRjhanou hed lhoral,

Tul'punthotyCOuanine merasthe the be warind emame,
Thisl t vowNGE mshithe t ved tis w wnolink as t s f'lisl

That cllst it woforue pie hrowi,
CIUusside,

Uheorougs Ohe pepoud me the hhe t cwiga poarsund p sNthee ichntkereag alouncotord
F hidgoncowodsist he ar s

?erad, y ss fro wapanoon o g.

prarton irend s re

CHure lourthit wanUu ,
B

ThimuraceDUCOu thathil, sou
A:
P my m.
Towathante amilere tonrouthe aposastarot G'paimusitouathe llicoo vnlegoseriset el
Aald t t uomurenacome herat'!
Wker asid mold wn t hed wyfoansugvoush, ar thou,
;


Whaldn, 

## Self-Attention

In [34]:
class Head(nn.Module):

    key: nn.Linear
    query: nn.Linear
    value: nn.Linear

    def __init__(self, head_size: int, n_embd: int, block_size:int , device = None):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False, device=device)
        self.query = nn.Linear(n_embd, head_size, bias=False, device=device)
        self.value = nn.Linear(n_embd, head_size, bias=False, device=device)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size, device=device)))

    def forward(self, x: Tensor) -> Tensor:
        B, T, C = x.shape

        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)

        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        v = self.value(x) # (B, T, C)
        out = wei @ v # (B, T, C)

        return out

In [35]:
class BigramLanguageModel(nn.Module):

    vocab_size: int
    n_embd: int
    block_size: int
    head_size: int

    token_embedding_table: nn.Embedding
    position_embedding_table: nn.Embedding
    lm_head: nn.Linear
    sa_head: Head

    def __init__(self, vocab_size: int, block_size: int, n_embd: int, head_size: int, device=None):
        super().__init__()

        self.vocab_size = vocab_size
        self.n_embd = n_embd

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)

        self.sa_head = Head(head_size, n_embd, block_size, device=device)
        self.lm_head = nn.Linear(n_embd, vocab_size, device=device)

    def forward(self, idx: Tensor, targets: typing.Optional[Tensor] = None) -> typing.Tuple[Tensor, typing.Optional[Tensor]]:
        # `idx` and targets are (B,T) tensors (batch size by time). In this case
        # 'time' represents block size.
        B, T = idx.shape

        # `logits` are (B,T,C) tensors, (batch size by time by channel), where
        # the channel dimension comes from the embedding table. Essentially,
        # each character in idx is replaced by an embedding vector of length C
        # (which is the number of embeddings in this case).
        token_embeddings = self.token_embedding_table(idx)

        # Shape is (T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device))

        # Addition gets broadcasted, shape is (B,T,C)
        x = token_embeddings + position_embeddings

        # Apply self-attention head
        x = self.sa_head(x)

        # We then apply the linear layer, which gives us a (B, T, vocab_size)
        # tensor, which are our logits.
        logits = self.lm_head(x)

        logits = typing.cast(Tensor, logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets)

        # If `targets` was not provided, then output `logits` is a 3D tensor of
        # shape:
        #     `(batch_size, block_size, vocab_size)`
        #
        # Otherwise, if `targets` was provided, then output `logits` is a 2D
        # tensor of shape:
        #     `(batch_size * block_size, vocab_size)`
        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # `idx` is (B,T), which is `(batch_size, block_size)`
        for _ in range(max_new_tokens):
            # Crop idx to the last `block_size` tokens
            idx_cond = idx[:, -block_size:]
            # `logits` is (B,T,C), where C is the channel length (length of
            # embedding vector, in this case it is `vocab_length`)
            logits, loss = self(idx_cond)
            # Get last character of logits - becomes (B, C)
            logits = logits[:, -1, :]
            # Still (B,C)
            probs = F.softmax(logits, dim=-1)
            # Now its (B,1) since we are getting only one sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append sampled index to the running sequence - becomes (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)
        # The final `idx` tensor will be of shape
        #     `(batch_size, block_size + max_steps)`
        return idx

In [36]:
batch_size = 32
max_steps = 1000
learning_rate = 1e-3
eval_iterations = 300
n_embd = 32
head_size = 32

model = BigramLanguageModel(vocab_size, block_size, n_embd, head_size, device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.train()

for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    model.eval()
    if max_steps < 25 or step % (max_steps // 25) == 0:
        loss_dict = estimate_loss(model, train_data, val_data, eval_iterations, batch_size, block_size, device)
        print(f'Step: {step:<7}, last seen loss: {loss.item():.4f}, estimated training loss: {loss_dict["train"]:.4f}, estimated validation loss: {loss_dict["val"]:.4f}')

    model.train()

Step: 0      , last seen loss: 4.2917, estimated training loss: 4.2655, estimated validation loss: 4.2675
Step: 40     , last seen loss: 3.5896, estimated training loss: 3.5140, estimated validation loss: 3.5109
Step: 80     , last seen loss: 3.1798, estimated training loss: 3.2542, estimated validation loss: 3.2502
Step: 120    , last seen loss: 3.2870, estimated training loss: 3.1639, estimated validation loss: 3.1561
Step: 160    , last seen loss: 3.0641, estimated training loss: 3.0917, estimated validation loss: 3.0901
Step: 200    , last seen loss: 2.9114, estimated training loss: 3.0270, estimated validation loss: 3.0155
Step: 240    , last seen loss: 2.9914, estimated training loss: 2.9574, estimated validation loss: 2.9502
Step: 280    , last seen loss: 2.9581, estimated training loss: 2.8989, estimated validation loss: 2.8841
Step: 320    , last seen loss: 2.8008, estimated training loss: 2.8410, estimated validation loss: 2.8312
Step: 360    , last seen loss: 2.8981, estimat

In [37]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=1000)[0].tolist()
next_str = decode(next_idx)
print(next_str)


SE: cou Wheyulaw he s ie, be fse nggth ol d
KE:
SINO? s murdatalistous 's it.
Y lher ithaillsit, thif theas wh buthens.Gsere s'd rirtho ss t, lothe cisrsgeserser Whe t thiced hek fez' sonen lo nd, myimrinrve ote:
Lou irech!
ig y my ferseilde ort ILen:
Herulo
Y slve mys th stan he te sekay, ant argive it.
NCse k n I Fthast ce trane ad pis'dey wwower fofrind itainepe teaend cuges mevear thas nlixn f isensur'g ty o le,
AUS:
Tour,
SA, ln deurr k t, tun'dor denriro atopayore stowos
Whe sK:
ACBfr d,
Altds
Oissthe anwerd me, tha alid tis arhur-mo y athingseersKTitits bours heerefo! orcovem d ou lathureas toh upeeecr, p, nad.

CINUKD
K;
powo te LI
COLTAnowuetethes en ithond, bave,
Hy ICwheas nse.

The fac wathe preandri
S f bais.

Y:
I qy ISa.
SToor he ssoucilweal thisipl mesc thads and athe indieminpooneotos
CEET:
KLAngyo-te:
UBTLENI amis'd seat
bol qive
Acforeayi, low Hard gangige ce t moun sst ve am,hrit walk ing to wat, bauthofimad d dse heitat hoad hd brareaco meint miond overrtat y;ino 

### Multiple-Head Attention

In [38]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads: int, head_size: int, n_embd: int, block_size: int, device = None):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size, device=device) for _ in range(num_heads)])

    def forward(self, x: Tensor) -> Tensor:
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [39]:
class BigramLanguageModel(nn.Module):

    vocab_size: int
    n_embd: int
    block_size: int
    head_size: int

    token_embedding_table: nn.Embedding
    position_embedding_table: nn.Embedding
    lm_head: nn.Linear
    sa_heads: MultiHeadAttention

    def __init__(self, vocab_size: int, block_size: int, n_embd: int, num_heads: int, head_size: int, device=None):
        super().__init__()

        self.vocab_size = vocab_size
        self.n_embd = n_embd

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)

        self.sa_heads = MultiHeadAttention(num_heads, head_size // num_heads, n_embd, block_size, device=device)
        self.lm_head = nn.Linear(n_embd, vocab_size, device=device)

    def forward(self, idx: Tensor, targets: typing.Optional[Tensor] = None) -> typing.Tuple[Tensor, typing.Optional[Tensor]]:
        # `idx` and targets are (B,T) tensors (batch size by time). In this case
        # 'time' represents block size.
        B, T = idx.shape

        # `logits` are (B,T,C) tensors, (batch size by time by channel), where
        # the channel dimension comes from the embedding table. Essentially,
        # each character in idx is replaced by an embedding vector of length C
        # (which is the number of embeddings in this case).
        token_embeddings = self.token_embedding_table(idx)

        # Shape is (T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device))

        # Addition gets broadcasted, shape is (B,T,C)
        x = token_embeddings + position_embeddings

        # Apply self-attention head
        x = self.sa_heads(x)

        # We then apply the linear layer, which gives us a (B, T, vocab_size)
        # tensor, which are our logits.
        logits = self.lm_head(x)

        logits = typing.cast(Tensor, logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets)

        # If `targets` was not provided, then output `logits` is a 3D tensor of
        # shape:
        #     `(batch_size, block_size, vocab_size)`
        #
        # Otherwise, if `targets` was provided, then output `logits` is a 2D
        # tensor of shape:
        #     `(batch_size * block_size, vocab_size)`
        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # `idx` is (B,T), which is `(batch_size, block_size)`
        for _ in range(max_new_tokens):
            # Crop idx to the last `block_size` tokens
            idx_cond = idx[:, -block_size:]
            # `logits` is (B,T,C), where C is the channel length (length of
            # embedding vector, in this case it is `vocab_length`)
            logits, loss = self(idx_cond)
            # Get last character of logits - becomes (B, C)
            logits = logits[:, -1, :]
            # Still (B,C)
            probs = F.softmax(logits, dim=-1)
            # Now its (B,1) since we are getting only one sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append sampled index to the running sequence - becomes (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)
        # The final `idx` tensor will be of shape
        #     `(batch_size, block_size + max_steps)`
        return idx

In [40]:
batch_size = 32
max_steps = 1000
learning_rate = 1e-3
eval_iterations = 300
n_embd = 32
num_heads = 4
head_size = 32

model = BigramLanguageModel(vocab_size, block_size, n_embd, num_heads, head_size, device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.train()

for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    model.eval()
    if max_steps < 25 or step % (max_steps // 25) == 0:
        loss_dict = estimate_loss(model, train_data, val_data, eval_iterations, batch_size, block_size, device)
        print(f'Step: {step:<7}, last seen loss: {loss.item():.4f}, estimated training loss: {loss_dict["train"]:.4f}, estimated validation loss: {loss_dict["val"]:.4f}')

    model.train()

Step: 0      , last seen loss: 4.2198, estimated training loss: 4.2017, estimated validation loss: 4.2007
Step: 40     , last seen loss: 3.6347, estimated training loss: 3.5558, estimated validation loss: 3.5525
Step: 80     , last seen loss: 3.2789, estimated training loss: 3.2495, estimated validation loss: 3.2456
Step: 120    , last seen loss: 3.1077, estimated training loss: 3.1416, estimated validation loss: 3.1338
Step: 160    , last seen loss: 2.9623, estimated training loss: 3.0601, estimated validation loss: 3.0621
Step: 200    , last seen loss: 3.0927, estimated training loss: 2.9884, estimated validation loss: 2.9756
Step: 240    , last seen loss: 3.0324, estimated training loss: 2.9203, estimated validation loss: 2.8969
Step: 280    , last seen loss: 2.7927, estimated training loss: 2.8574, estimated validation loss: 2.8317
Step: 320    , last seen loss: 2.8187, estimated training loss: 2.7929, estimated validation loss: 2.7887
Step: 360    , last seen loss: 2.7457, estimat

In [41]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=1000)[0].tolist()
next_str = decode(next_idx)
print(next_str)


IHLREMILIt say thalt cais aseend thomise?

Cand in.

coud,
TY:
The A
L3ideasafaned; deavs withe t,
cil ky!s thhiit ho dor
Pourefradke thisooncin;,
H I.

TEE
Lon va? Iwer; fas. fod.
ENThe uso yosrk Igorerom melisof rebrreiroouse shyaaang!
Ich!
Wins Rernd no, cais cas jhy is ben'se se k d sow:
Fof'dtert uths bry fadonond Mrarevrt he ivin
Qtheicen, se won hres und le sthey
e tan.
OY longrfe, yetod tho the fe pre dinuerifhellt toum fheiqo'lk thane sistas
A;
LIt san' nghor ifl sacms,, Gomt an enowet
Y lory, ofunvn an m! swir cfne shans u ckslerm, byavoruns yormus hast,
Hwey binghhist

yrs tam does mYib ryoerepweit fer mirmecat yot go thor xor-nt dorles ath, ild ge.

Q:
At yoyoontouve ha ore
Po ancinler, thave oof loin thant e asishon he rinddetde ou ras.

AmHe IUACO bhe mayf hIyer, bthe
Ane,
S
ADFy induge arn't:
I nndeciy?
Norr thacid, kechd asit them thend so meynt gayt
yer thumy't, nak, soany hy wakts,
Thand&.

: fondon,

went, Irue,
Man allcthps.

he dat,
The gnf ou?
 thinde rtt aagepsi

### Adding Feed-Forward Layer

This layer adds computation to the communication between tokens.

In [42]:
class FeedForward(nn.Module):

    def __init__(self, n_embd: int, device = None):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd, device=device),
            nn.ReLU()
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.net(x)

In [43]:
class BigramLanguageModel(nn.Module):

    vocab_size: int
    n_embd: int
    block_size: int
    head_size: int

    token_embedding_table: nn.Embedding
    position_embedding_table: nn.Embedding
    lm_head: nn.Linear
    ffwd: FeedForward
    sa_heads: MultiHeadAttention

    def __init__(self, vocab_size: int, block_size: int, n_embd: int, num_heads: int, head_size: int, device=None):
        super().__init__()

        self.vocab_size = vocab_size
        self.n_embd = n_embd

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)

        self.sa_heads = MultiHeadAttention(num_heads, head_size // num_heads, n_embd, block_size, device=device)
        self.ffwd = FeedForward(n_embd, device=device)
        self.lm_head = nn.Linear(n_embd, vocab_size, device=device)

    def forward(self, idx: Tensor, targets: typing.Optional[Tensor] = None) -> typing.Tuple[Tensor, typing.Optional[Tensor]]:
        # `idx` and targets are (B,T) tensors (batch size by time). In this case
        # 'time' represents block size.
        B, T = idx.shape

        # `logits` are (B,T,C) tensors, (batch size by time by channel), where
        # the channel dimension comes from the embedding table. Essentially,
        # each character in idx is replaced by an embedding vector of length C
        # (which is the number of embeddings in this case).
        token_embeddings = self.token_embedding_table(idx)

        # Shape is (T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device))

        # Addition gets broadcasted, shape is (B,T,C)
        x = token_embeddings + position_embeddings

        # Apply self-attention head
        x = self.sa_heads(x)

        # We then apply the linear layer, which gives us a (B, T, vocab_size)
        # tensor, which are our logits.
        logits = self.lm_head(x)

        logits = typing.cast(Tensor, logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets)

        # If `targets` was not provided, then output `logits` is a 3D tensor of
        # shape:
        #     `(batch_size, block_size, vocab_size)`
        #
        # Otherwise, if `targets` was provided, then output `logits` is a 2D
        # tensor of shape:
        #     `(batch_size * block_size, vocab_size)`
        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # `idx` is (B,T), which is `(batch_size, block_size)`
        for _ in range(max_new_tokens):
            # Crop idx to the last `block_size` tokens
            idx_cond = idx[:, -block_size:]
            # `logits` is (B,T,C), where C is the channel length (length of
            # embedding vector, in this case it is `vocab_length`)
            logits, loss = self(idx_cond)
            # Get last character of logits - becomes (B, C)
            logits = logits[:, -1, :]
            # Still (B,C)
            probs = F.softmax(logits, dim=-1)
            # Now its (B,1) since we are getting only one sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append sampled index to the running sequence - becomes (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)
        # The final `idx` tensor will be of shape
        #     `(batch_size, block_size + max_steps)`
        return idx

In [44]:
batch_size = 32
max_steps = 1000
learning_rate = 1e-3
eval_iterations = 300
n_embd = 32
num_heads = 4
head_size = 32

model = BigramLanguageModel(vocab_size, block_size, n_embd, num_heads, head_size, device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.train()

for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    model.eval()
    if max_steps < 25 or step % (max_steps // 25) == 0:
        loss_dict = estimate_loss(model, train_data, val_data, eval_iterations, batch_size, block_size, device)
        print(f'Step: {step:<7}, last seen loss: {loss.item():.4f}, estimated training loss: {loss_dict["train"]:.4f}, estimated validation loss: {loss_dict["val"]:.4f}')

    model.train()

Step: 0      , last seen loss: 4.2541, estimated training loss: 4.2554, estimated validation loss: 4.2507
Step: 40     , last seen loss: 3.5468, estimated training loss: 3.5652, estimated validation loss: 3.5602
Step: 80     , last seen loss: 3.2690, estimated training loss: 3.2291, estimated validation loss: 3.2266
Step: 120    , last seen loss: 3.0321, estimated training loss: 3.1367, estimated validation loss: 3.1349
Step: 160    , last seen loss: 3.0168, estimated training loss: 3.0726, estimated validation loss: 3.0545
Step: 200    , last seen loss: 2.9816, estimated training loss: 2.9856, estimated validation loss: 2.9778
Step: 240    , last seen loss: 3.0848, estimated training loss: 2.9134, estimated validation loss: 2.9011
Step: 280    , last seen loss: 2.8154, estimated training loss: 2.8645, estimated validation loss: 2.8471
Step: 320    , last seen loss: 2.7762, estimated training loss: 2.8158, estimated validation loss: 2.8140
Step: 360    , last seen loss: 2.7335, estimat

In [45]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=1000)[0].tolist()
next_str = decode(next_idx)
print(next_str)


Goumeither ser forhy to; airr,

Mh:
ENRT, pyime sndis, o d haspind.

CAs;
U&meelt:
Dbe n;
A-RW
E
OR-W
DM.
KIRK'wes cheilng cucow hargante bid tyE, Oy beo'ou hes wes.
M F fr.
Bmne.
Ke,
Th
Tou ne
Mesore byod tatiropkle.
UIn.
YHin'g t sthy orwor alprlTavees my here lspomtos shamand, annt shise I ma.
AL
W
R's.
Theane sholn. weavasis heklim la am t'th laes Qrledeit y bliremll'ther ache neneth let ha, Ausony her fver'arisind malerl lacsesh thar buso wwe fhil ovolilng I M:
T Binias f, whay 'y
Jm antiy whed malocanot'd sherch hesdeuse c,
Rer

WAthe Othi; tethich.
WOLh ses.
Be, L
MI, eungeil-aldend.
M
LG'M?
MEBIN
TELINIUIINCF: wou nisre irrt yul atha?
LAv
Wh fenes'l thee choumer, sooon
ow fledk thher sofst this ger the ithe map I mal'dy;
JYhone, R barul,
Han hisicot te o amba bir de sangitrigh oee mang spt ren weo genoaurmr ape tyeshaighet gy, gdalerel igofove ould mapo:
Ser rele ndd, o.
W

IILOnss qje
Lie ndoverl
Lh
Wth t barbth;et as?
T o fre wo orf batheaveshelor far joy I mAfe, lxrou lekp 

### Skip Connections, Layer Normalization, and Grouping Communication and Computation Layers into Blocks

In [52]:
class LayerNorm(nn.Module):

    eps: int
    gamma: Tensor
    beta: Tensor

    def  __init__(self, dim: int, eps: float = 1e-5, device = None):
        self.eps = eps
        self.gamma = torch.ones(dim, device=device)
        self.beta = torch.zeros(dim, device=device)

    def __call__(self, x: Tensor) -> Tensor:
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self) -> typing.List[Tensor]:
        return [self.gamma, self.beta]

In [53]:
class MultiHeadAttention(nn.Module):

    heads: nn.ModuleList
    projection: nn.Linear

    def __init__(self, num_heads: int, head_size: int, n_embd: int, block_size: int, device = None):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size, device=device) for _ in range(num_heads)])
        self.projection = nn.Linear(num_heads * head_size, n_embd, device=device)

    def forward(self, x: Tensor) -> Tensor:
        # Apply the self-attention heads
        out = torch.cat([h(x) for h in self.heads], dim=-1)

        # Apply the projection
        out = self.projection(out)

        return out

In [54]:
class FeedForward(nn.Module):

    def __init__(self, n_embd: int, device = None):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd * 4, device=device),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd, device=device) # Projection layer
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.net(x)

In [55]:
class Block(nn.Module):

    sa: MultiHeadAttention
    ffwd: FeedForward
    ln1 = LayerNorm
    ln2 = LayerNorm

    def __init__(self, n_embd: int, num_heads: int, block_size: int, device = None):
        super().__init__()
        print(device)
        head_size = n_embd // num_heads
        self.sa = MultiHeadAttention(num_heads, head_size, n_embd, block_size, device=device)
        self.ffwd = FeedForward(n_embd, device=device)
        self.ln1 = LayerNorm(n_embd, device=device)
        self.ln2 = LayerNorm(n_embd, device=device)

    def forward(self, x: Tensor) -> Tensor:
        # Adding `x` to the layers is the skip connection
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [56]:
class BigramLanguageModel(nn.Module):

    vocab_size: int
    n_embd: int
    block_size: int
    head_size: int

    token_embedding_table: nn.Embedding
    position_embedding_table: nn.Embedding
    blocks: nn.Sequential
    lm_head: nn.Linear

    def __init__(self, vocab_size: int, block_size: int, n_embd: int, num_heads: int, head_size: int, device=None):
        super().__init__()

        self.vocab_size = vocab_size
        self.n_embd = n_embd

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)

        self.blocks = nn.Sequential(
            Block(n_embd, num_heads, block_size, device=device),
            Block(n_embd, num_heads, block_size, device=device),
            Block(n_embd, num_heads, block_size, device=device),
            LayerNorm(n_embd, device=device)
        )

        self.lm_head = nn.Linear(n_embd, vocab_size, device=device)

    def forward(self, idx: Tensor, targets: typing.Optional[Tensor] = None) -> typing.Tuple[Tensor, typing.Optional[Tensor]]:
        # `idx` and targets are (B,T) tensors (batch size by time). In this case
        # 'time' represents block size.
        B, T = idx.shape

        # `logits` are (B,T,C) tensors, (batch size by time by channel), where
        # the channel dimension comes from the embedding table. Essentially,
        # each character in idx is replaced by an embedding vector of length C
        # (which is the number of embeddings in this case).
        token_embeddings = self.token_embedding_table(idx)

        # Shape is (T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device))

        # Addition gets broadcasted, shape is (B,T,C)
        x = token_embeddings + position_embeddings

        # Apply self-attention head
        x = self.blocks(x)

        # We then apply the linear layer, which gives us a (B, T, vocab_size)
        # tensor, which are our logits.
        logits = self.lm_head(x)

        logits = typing.cast(Tensor, logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets)

        # If `targets` was not provided, then output `logits` is a 3D tensor of
        # shape:
        #     `(batch_size, block_size, vocab_size)`
        #
        # Otherwise, if `targets` was provided, then output `logits` is a 2D
        # tensor of shape:
        #     `(batch_size * block_size, vocab_size)`
        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # `idx` is (B,T), which is `(batch_size, block_size)`
        for _ in range(max_new_tokens):
            # Crop idx to the last `block_size` tokens
            idx_cond = idx[:, -block_size:]
            # `logits` is (B,T,C), where C is the channel length (length of
            # embedding vector, in this case it is `vocab_length`)
            logits, loss = self(idx_cond)
            # Get last character of logits - becomes (B, C)
            logits = logits[:, -1, :]
            # Still (B,C)
            probs = F.softmax(logits, dim=-1)
            # Now its (B,1) since we are getting only one sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append sampled index to the running sequence - becomes (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)
        # The final `idx` tensor will be of shape
        #     `(batch_size, block_size + max_steps)`
        return idx

In [57]:
batch_size = 32
max_steps = 1000
learning_rate = 1e-3
eval_iterations = 300
n_embd = 32
num_heads = 4
head_size = 32

model = BigramLanguageModel(vocab_size, block_size, n_embd, num_heads, head_size, device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.train()

for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size=batch_size, block_size=block_size, device=device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    model.eval()
    if max_steps < 25 or step % (max_steps // 25) == 0:
        loss_dict = estimate_loss(model, train_data, val_data, eval_iterations, batch_size, block_size, device)
        print(f'Step: {step:<7}, last seen loss: {loss.item():.4f}, estimated training loss: {loss_dict["train"]:.4f}, estimated validation loss: {loss_dict["val"]:.4f}')

    model.train()

cuda
cuda
cuda


AttributeError: 'LayerNorm' object has no attribute '_parameters'

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
next_idx = model.generate(idx, max_new_tokens=1000)[0].tolist()
next_str = decode(next_idx)
print(next_str)

hi
kllnlgo
2
go
hi
kllnlgo
2
go
hi
kllnlgo
2
go
hi
kllnlgo
2


/tmp/ipykernel_44024/1069408570.py:14: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/aten/src/ATen/native/ReduceOps.cpp:1823.)
  xvar = x.var(1, keepdim=True)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


/opt/conda/conda-bld/pytorch_1729647378361/work/aten/src/ATen/native/cuda/TensorCompare.cu:110: _assert_async_cuda_kernel: block: [0,0,0], thread: [0,0,0] Assertion `probability tensor contains either `inf`, `nan` or element < 0` failed.
